In [4]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import zipfile

np.random.seed(42)
random.seed(42)

# -----------------------------
# CONFIGURATION
# -----------------------------
NUM_CUSTOMERS = 20000
START_DATE = datetime(2022, 1, 1)
END_DATE = datetime(2025, 12, 31)

channels = ["Google Ads", "Meta Ads", "Organic", "Referral", "Local Promo", "Fleet", "Fundraising"]
channel_weights = [0.25, 0.20, 0.18, 0.12, 0.10, 0.10, 0.05]

membership_types = ["None", "Basic", "Premium", "Unlimited"]
membership_prices = {"Basic": 19.99, "Premium": 29.99, "Unlimited": 39.99}

cities = ["Birmingham", "Atlanta", "Nashville", "Dallas", "Orlando", "Charlotte"]
states = ["AL", "GA", "TN", "TX", "FL", "NC"]

vehicle_types = ["Sedan", "SUV", "Truck", "Van"]
income_bands = ["<40k", "40-75k", "75-100k", "100k+"]

# Channel-based behavior (retention & value)
channel_ltv_multiplier = {
    "Google Ads": 1.2,
    "Meta Ads": 0.9,
    "Organic": 1.4,
    "Referral": 1.3,
    "Local Promo": 0.7,
    "Fleet": 1.6,
    "Fundraising": 0.8
}

# -----------------------------
# HELPER FUNCTIONS
# -----------------------------
def random_date(start, end):
    return start + timedelta(days=random.randint(0, (end - start).days))

def seasonal_multiplier(date):
    # Higher demand in summer months
    if date.month in [5,6,7,8]:
        return 1.3
    elif date.month in [12,1]:
        return 0.8
    return 1.0

# -----------------------------
# 1) CUSTOMERS TABLE
# -----------------------------
customers = []

for i in range(NUM_CUSTOMERS):
    signup_date = random_date(START_DATE, END_DATE)
    channel = np.random.choice(channels, p=channel_weights)
    membership = np.random.choice(membership_types, p=[0.4, 0.25, 0.2, 0.15])

    city_idx = random.randint(0, len(cities)-1)

    customers.append({
        "customer_id": f"C{i:06d}",
        "signup_date": signup_date.date(),
        "signup_channel": channel,
        "campaign": f"Campaign_{random.randint(1,12)}",
        "location_id": f"L{random.randint(1,30):03d}",
        "city": cities[city_idx],
        "state": states[city_idx],
        "membership_type": membership,
        "age": random.randint(18,70),
        "income_band": random.choice(income_bands),
        "vehicle_type": random.choice(vehicle_types)
    })

customers_df = pd.DataFrame(customers)

# -----------------------------
# 2) TRANSACTIONS TABLE
# -----------------------------
transactions = []

for _, c in customers_df.iterrows():
    base_date = pd.to_datetime(c["signup_date"])

    # churn probability depends on channel & membership
    churn_prob = 0.15 / channel_ltv_multiplier[c["signup_channel"]]

    active_months = random.randint(1, 36)

    for m in range(active_months):
        month_date = base_date + pd.DateOffset(months=m)
        if month_date > END_DATE:
            break

        # membership fee
        if c["membership_type"] != "None":
            if random.random() > churn_prob:
                transactions.append({
                    "transaction_id": f"T{random.randint(1000000,9999999)}",
                    "customer_id": c["customer_id"],
                    "transaction_date": month_date.date(),
                    "location_id": c["location_id"],
                    "service_type": "membership_fee",
                    "revenue": membership_prices[c["membership_type"]],
                    "payment_type": random.choice(["card","card","card","cash"])
                })

        # extra visits / single washes
        extra_visits = np.random.poisson(1.2 * seasonal_multiplier(month_date))
        for _ in range(extra_visits):
            tx_date = month_date + timedelta(days=random.randint(0,27))
            if tx_date > END_DATE:
                break
            transactions.append({
                "transaction_id": f"T{random.randint(1000000,9999999)}",
                "customer_id": c["customer_id"],
                "transaction_date": tx_date.date(),
                "location_id": c["location_id"],
                "service_type": np.random.choice(["single_wash","upsell"], p=[0.7,0.3]),
                "revenue": round(random.uniform(8,25),2),
                "payment_type": random.choice(["card","cash"])
            })

transactions_df = pd.DataFrame(transactions)

# -----------------------------
# 3) MARKETING SPEND TABLE
# -----------------------------
marketing_rows = []
dates = pd.date_range(START_DATE, END_DATE)

for d in dates:
    for channel in ["Google Ads","Meta Ads"]:
        base_spend = random.uniform(800, 6000)
        spend = round(base_spend * seasonal_multiplier(d),2)
        impressions = int(random.uniform(20000, 120000))
        clicks = int(impressions * random.uniform(0.01,0.05))
        conversions = int(clicks * random.uniform(0.02,0.12))

        marketing_rows.append({
            "date": d.date(),
            "channel": channel,
            "campaign": f"Campaign_{random.randint(1,12)}",
            "ad_group": f"AdGroup_{random.randint(1,6)}",
            "spend": spend,
            "impressions": impressions,
            "clicks": clicks,
            "conversions": conversions
        })

marketing_df = pd.DataFrame(marketing_rows)

# -----------------------------
# 4) CRM EVENTS TABLE
# -----------------------------
crm_events = []

for _, c in customers_df.sample(frac=0.7).iterrows():
    num_events = random.randint(1,6)
    for _ in range(num_events):
        event_type = np.random.choice(
            ["email_open","sms_click","app_login","churn","reactivation"],
            p=[0.35,0.2,0.25,0.15,0.05]
        )
        event_date = pd.to_datetime(c["signup_date"]) + timedelta(days=random.randint(1,900))
        if event_date > END_DATE:
            continue

        crm_events.append({
            "event_id": f"E{random.randint(1000000,9999999)}",
            "customer_id": c["customer_id"],
            "event_type": event_type,
            "event_date": event_date.date(),
            "channel": random.choice(["email","sms","app","system"])
        })

crm_df = pd.DataFrame(crm_events)

# -----------------------------
# 5) CUSTOMER SERVICE TABLE
# -----------------------------
service_rows = []

for _, c in customers_df.sample(frac=0.45).iterrows():
    service_rows.append({
        "ticket_id": f"S{random.randint(1000000,9999999)}",
        "customer_id": c["customer_id"],
        "issue_type": random.choice(["Billing","Service Quality","App Issue","Membership Cancellation"]),
        "resolution_time_minutes": random.randint(5,240),
        "csat_score": random.randint(1,5)
    })

service_df = pd.DataFrame(service_rows)

# -----------------------------
# 6) NPS REVIEWS TABLE
# -----------------------------
reviews = []

for _, c in customers_df.sample(frac=0.35).iterrows():
    score = random.randint(0,10)
    sentiment = "positive" if score >= 8 else "neutral" if score >=5 else "negative"

    reviews.append({
        "review_id": f"R{random.randint(1000000,9999999)}",
        "customer_id": c["customer_id"],
        "review_date": (pd.to_datetime(c["signup_date"]) + timedelta(days=random.randint(10,700))).date(),
        "nps_score": score,
        "sentiment": sentiment
    })

nps_df = pd.DataFrame(reviews)

# -----------------------------
# EXPORT CSV FILES
# -----------------------------
customers_df.to_csv("customers.csv", index=False)
transactions_df.to_csv("transactions.csv", index=False)
marketing_df.to_csv("marketing_spend.csv", index=False)
crm_df.to_csv("crm_events.csv", index=False)
service_df.to_csv("customer_service.csv", index=False)
nps_df.to_csv("nps_reviews.csv", index=False)

# -----------------------------
# ZIP FILE FOR DOWNLOAD
# -----------------------------
file_list = [
    "customers.csv",
    "transactions.csv",
    "marketing_spend.csv",
    "crm_events.csv",
    "customer_service.csv",
    "nps_reviews.csv"
]

with zipfile.ZipFile("tidal_wave_dataset.zip", "w") as zipf:
    for file in file_list:
        zipf.write(file)

print("✅ Dataset generated successfully!")
print("Files created:", file_list)



✅ Dataset generated successfully!
Files created: ['customers.csv', 'transactions.csv', 'marketing_spend.csv', 'crm_events.csv', 'customer_service.csv', 'nps_reviews.csv']


In [6]:
from google.colab import files
files.download("tidal_wave_dataset.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>